# Extract and Transform Data

#### In this notebook we will be extracting movie related information from the OMDB API and save a csv file.

1. Data source:
2. API Documentation:

In [66]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
import json
import random
from pprint import pprint
from config import LOCAL_API_KEY
import time

In [15]:
# Visualize more columns
pd.options.display.max_columns = None

## Initial clean of the downloaded CSV

In [12]:
# Import downloaded CSV which contains a list 6820 movies
file_path = '../resources/raw_data/movies.csv'
kaggle_df = pd.read_csv(file_path, sep=',', engine='python')
kaggle_df.head()

,budget,company,country,director,genre,gross,name,rating,released,runtime,score,star,votes,writer,year
0,8000000.0,Columbia Pictures Corporation,USA,Rob Reiner,Adventure,52287414.0,Stand by Me,R,1986-08-22,89,8.1,Wil Wheaton,299174,Stephen King,1986
1,6000000.0,Paramount Pictures,USA,John Hughes,Comedy,70136369.0,Ferris Bueller's Day Off,PG-13,1986-06-11,103,7.8,Matthew Broderick,264740,John Hughes,1986
2,15000000.0,Paramount Pictures,USA,Tony Scott,Action,179800601.0,Top Gun,PG,1986-05-16,110,6.9,Tom Cruise,236909,Jim Cash,1986
3,18500000.0,Twentieth Century Fox Film Corporation,USA,James Cameron,Action,85160248.0,Aliens,R,1986-07-18,137,8.4,Sigourney Weaver,540152,James Cameron,1986
4,9000000.0,Walt Disney Pictures,USA,Randal Kleiser,Adventure,18564613.0,Flight of the Navigator,PG,1986-08-01,90,6.9,Joey Cramer,36636,Mark H. Baker,1986


In [42]:
# Describe
kaggle_df.describe()

,budget,gross,runtime,score,votes,year
count,6.820000e+03,6.820000e+03,6820.00000,6820.000000,6.820000e+03,6820.000000
mean,2.458113e+07,3.349783e+07,106.55132,6.374897,7.121952e+04,2001.000293
std,3.702254e+07,5.819760e+07,18.02818,1.003142,1.305176e+05,8.944501
min,0.000000e+00,7.000000e+01,50.00000,1.500000,2.700000e+01,1986.000000
25%,0.000000e+00,1.515839e+06,95.00000,5.800000,7.665250e+03,1993.000000
50%,1.100000e+07,1.213568e+07,102.00000,6.400000,2.589250e+04,2001.000000
75%,3.200000e+07,4.006534e+07,115.00000,7.100000,7.581225e+04,2009.000000
max,3.000000e+08,9.366622e+08,366.00000,9.300000,1.861666e+06,2016.000000


In [44]:
# Number of rows
len(kaggle_df)

6820

In [27]:
# Rename a few columns to make distinctions between the Kaggle CSV and dimensions
# scraped from the OMDB API
renamed_df = kaggle_df.rename(columns={'country': 'country_kaggle',
                                       'genre': 'genre_kaggle',
                                       'writer': 'writer_kaggle',
                                       'star': 'star_kaggle',
                                       'company': 'production',
                                       'score': 'score_imdb',
                                       'votes': 'votes_imdb'})
renamed_df.head(2)

,budget,production,country_kaggle,director,genre_kaggle,gross,name,rating,released,runtime,score_imdb,star_kaggle,votes_imdb,writer_kaggle,year
0,8000000.0,Columbia Pictures Corporation,USA,Rob Reiner,Adventure,52287414.0,Stand by Me,R,1986-08-22,89,8.1,Wil Wheaton,299174,Stephen King,1986
1,6000000.0,Paramount Pictures,USA,John Hughes,Comedy,70136369.0,Ferris Bueller's Day Off,PG-13,1986-06-11,103,7.8,Matthew Broderick,264740,John Hughes,1986


In [35]:
scraped_df = renamed_df.copy()

# Add empty columns for dimensions we are going to scrape from OMDB API
scraped_df['genres_omdb'] = ''
scraped_df['writers_omdb'] = ''
scraped_df['actors_omdb'] = ''
scraped_df['plot'] = ''
scraped_df['language_omdb'] = ''
scraped_df['country_omdb'] = ''
scraped_df['awards'] = ''
scraped_df['poster'] = ''
scraped_df['score_metacritic'] = ''
scraped_df['type'] = ''

scraped_df.head(2)

,budget,production,country_kaggle,director,genre_kaggle,gross,name,rating,released,runtime,score_imdb,star_kaggle,votes_imdb,writer_kaggle,year,genres_omdb,writers_omdb,actors_omdb,plot,language_omdb,country_omdb,awards,poster,score_metacritic,type
0,8000000.0,Columbia Pictures Corporation,USA,Rob Reiner,Adventure,52287414.0,Stand by Me,R,1986-08-22,89,8.1,Wil Wheaton,299174,Stephen King,1986,,,,,,,,,,
1,6000000.0,Paramount Pictures,USA,John Hughes,Comedy,70136369.0,Ferris Bueller's Day Off,PG-13,1986-06-11,103,7.8,Matthew Broderick,264740,John Hughes,1986,,,,,,,,,,


## Perform API Calls to OMDB API

And save extracted information in the dataframe

In [62]:
# Set up API key for requests
api_key = LOCAL_API_KEY

In [133]:
# Use the lat/lng we recovered to identify airports

print('Beginning Data Retrieval...')
print('-----------------------------')
    
for index, row in scraped_df.iloc[2000:3000, :].iterrows():

    # Use a new movie name for each new api call
    movie_name = scraped_df.loc[index, 'name']

    # Use the search term: 'International Airport' and our lat/lng
    query_url = f'http://www.omdbapi.com/?apikey={api_key}&t={movie_name}&plot=full'

    # make request to url and conver to json
    api_data = requests.get(query_url).json()
    
    # Build some mechanisms to better 
    if index % 100 == 0:
            print(f'Processing Movie Index #{index}')
            
    if index % 250 == 0:
        time.sleep(10)
    
    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        scraped_df.loc[index, 'genres_omdb'] = api_data['Genre']
        scraped_df.loc[index, 'writers_omdb'] = api_data['Writer']
        scraped_df.loc[index, 'actors_omdb'] = api_data['Actors']
        scraped_df.loc[index, 'plot'] = api_data['Plot']
        scraped_df.loc[index, 'language_omdb'] = api_data['Language']
        scraped_df.loc[index, 'country_omdb'] = api_data['Country']
        scraped_df.loc[index, 'awards'] = api_data['Awards']
        scraped_df.loc[index, 'poster'] = api_data['Poster']
        scraped_df.loc[index, 'score_metacritic'] = api_data['Metascore']
        scraped_df.loc[index, 'type'] = api_data['Type']
        
    except (KeyError, IndexError):
        print('Missing field/result... skipping.')

print('-----------------------------')
print('Data Retrieval Complete.') 
print('-----------------------------')

Beginning Data Retrieval...
-----------------------------
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Processing Movie Index #1200
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Processing Movie Index #1300
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Processing Movie Index #1400
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Processing Movie Index #1500
Missing field/result... skipping.
Processing Movie Index #1600
Missing field/result... skipping.
Processing Movie Index #1700
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result

In [126]:
scraped_df['type'].value_counts()

          5688
movie     1109
series      23
Name: type, dtype: int64

In [137]:
# scraped_df.count()
# Strategy: scrape 1000 new rows each time
test_df = scraped_df.iloc[2000:3001, :]
test_df

,budget,production,country_kaggle,director,genre_kaggle,gross,name,rating,released,runtime,score_imdb,star_kaggle,votes_imdb,writer_kaggle,year,genres_omdb,writers_omdb,actors_omdb,plot,language_omdb,country_omdb,awards,poster,score_metacritic,type
2000,20000000.0,United Artists,USA,Iain Softley,Comedy,7564000.0,Hackers,PG-13,1995-09-15,107,6.2,Jonny Lee Miller,57821,Rafael Moreu,1995,,,,,,,,,,
2001,19000000.0,Don Simpson/Jerry Bruckheimer Films,USA,Michael Bay,Action,65807024.0,Bad Boys,R,1995-04-07,119,6.8,Will Smith,186965,George Gallo,1995,,,,,,,,,,
2002,90000000.0,Cinergi Pictures Entertainment,USA,John McTiernan,Action,100012499.0,Die Hard with a Vengeance,R,1995-05-19,128,7.6,Bruce Willis,317792,Jonathan Hensleigh,1995,,,,,,,,,,
2003,10000000.0,Universal Pictures,USA,Tamra Davis,Comedy,25588750.0,Billy Madison,PG-13,1995-02-10,89,6.4,Adam Sandler,112423,Tim Herlihy,1995,,,,,,,,,,
2004,175000000.0,Universal Pictures,USA,Kevin Reynolds,Action,88246220.0,Waterworld,PG-13,1995-07-28,135,6.1,Kevin Costner,153742,Peter Rader,1995,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2996,23000000.0,Fox 2000 Pictures,USA,Wayne Wang,Comedy,18653615.0,Anywhere But Here,PG-13,1999-11-12,114,6.2,Susan Sarandon,15423,Mona Simpson,1999,,,,,,,,,,
2997,13000000.0,Miramax,USA,Kevin Williamson,Comedy,8946237.0,Teaching Mrs. Tingle,PG-13,1999-08-20,96,5.2,Helen Mirren,17917,Kevin Williamson,1999,,,,,,,,,,
2998,0.0,Shooting Gallery,USA,Hampton Fancher,Crime,368808.0,The Minus Man,R,1999-10-08,111,6.2,Owen Wilson,5765,Lew McCreary,1999,,,,,,,,,,
2999,65000000.0,Walt Disney Pictures,USA,Donald Petrie,Comedy,36830057.0,My Favorite Martian,PG,1999-02-12,94,5.0,Christopher Lloyd,11414,John L. Greene,1999,,,,,,,,,,


In [132]:
# Testing...
# ........

# Request data
movie= 'The Five Heartbeats'
# movie= 'Jason Lives: Friday the 13th Part VI'
test_url = f'http://www.omdbapi.com/?apikey={api_key}&t={movie}&plot=full'
response = requests.get(test_url)
data = response.json()

pprint(data)

{'Error': 'Request limit reached!', 'Response': 'False'}


In [47]:
pprint(data)

{'Actors': 'Jeff Goldblum, Geena Davis, John Getz, Joy Boushel',
 'Awards': 'Won 1 Oscar. Another 6 wins & 10 nominations.',
 'BoxOffice': '$40,456,565',
 'Country': 'USA, UK, Canada',
 'DVD': '25 Nov 2015',
 'Director': 'David Cronenberg',
 'Genre': 'Drama, Horror, Sci-Fi',
 'Language': 'English',
 'Metascore': '79',
 'Plot': 'Seth Brundle (Jeff Goldblum), a brilliant but eccentric scientist '
         'attempts to woo investigative journalist Veronica Quaife (Geena '
         'Davis) by offering her a scoop on his latest research in the field '
         'of matter transportation, which against all the expectations of the '
         'scientific establishment have proved successful. Up to a point. '
         'Brundle thinks he has ironed out the last problem when he '
         'successfully transports a living creature, but when he attempts to '
         'teleport himself a fly enters one of the transmission booths, and '
         'Brundle finds he is a changed man. This Science-Gone-M

In [138]:
# Export new dataset to CSV
final_df = scraped_df.copy()
final_df.to_csv('../resources/cleaned_data/movies_complete.csv', index=False)